In [1]:
import getEPH
import functionsForModels
import make_dummy
import schoolYears
import categorize
import functionsForModels
import createVariables

import pandas as pd
#http://statsmodels.sourceforge.net/devel/examples/generated/example_wls.html
import numpy as np
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from statsmodels.iolib.table import (SimpleTable, default_txt_fmt)
np.random.seed(1024)
%matplotlib inline

/home/pipe/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
getEPH.getEPHdbf('t310')

In [2]:
data1 = pd.read_csv('data/cleanDatat310.csv')
data2 = categorize.categorize(data1)
data3 = schoolYears.schoolYears(data2)
data4 = createVariables.createVariables(data3)


In [4]:
data = data4.copy()

In [6]:
data.columns

Index([u'CODUSU', u'NRO_HOGAR', u'COMPONENTE', u'AGLOMERADO', u'PONDERA',
       u'familyRelation', u'female', u'age', u'schoolLevel', u'finishedYear',
       u'lastYear', u'activity', u'empCond', u'unempCond', u'ITF', u'IPCF',
       u'P47T', u'P21', u'primary', u'secondary', u'university', u'education',
       u'education2', u'age2', u'id', u'lnIncome', u'lnIncomeT'],
      dtype='object')

In [9]:
jobsAndIncome = (data.activity == 1) & (data.lnIncome > 1) & ((data.familyRelation == 1)|(data.familyRelation == 2))
cleanData = data.copy().loc[jobsAndIncome,
                              ['id',
                            'familyRelation',
                              'age',
                              'age2',
                              'female',
                              'education',
                              'education2',
                              'primary',
                              'secondary',
                              'university',
                              'P21',
                              'P47T',
                              'lnIncome',
                               u'lnIncomeT']]




In [10]:
cleanData.head()

,id,familyRelation,age,age2,female,education,education2,primary,secondary,university,P21,P47T,lnIncome,lnIncomeT
2,3078611,1,42,1764,0,7.0,49.0,7.0,0.0,0.0,3000,3000,8.006368,8.006368
3,3078611,2,44,1936,1,17.0,289.0,7.0,5.0,5.0,2800,2800,7.937375,7.937375
7,3082781,1,38,1444,0,17.0,289.0,7.0,5.0,5.0,5000,6000,8.517193,8.699515
8,3082781,2,28,784,1,17.0,289.0,7.0,5.0,5.0,4000,4000,8.294050,8.294050
9,3119371,1,63,3969,0,17.0,289.0,7.0,5.0,5.0,3800,3800,8.242756,8.242756


In [11]:
pivot = cleanData.pivot(index='id', columns='familyRelation')
pivot.head()

age          age2        female      education        \
familyRelation     1     2       1      2      1    2         1     2   
id                                                                      
1250971         57.0   NaN  3249.0    NaN    1.0  NaN      14.0   NaN   
1256691         61.0   NaN  3721.0    NaN    0.0  NaN       7.0   NaN   
1256892         31.0  23.0   961.0  529.0    0.0  1.0      12.0  12.0   
1261181         38.0  30.0  1444.0  900.0    0.0  1.0       7.0   7.0   
1261521         25.0  25.0   625.0  625.0    1.0  0.0      16.0  12.0   

               education2           ...    university          P21          \
familyRelation          1      2    ...             1    2       1       2   
id                                  ...                                      
1250971             196.0    NaN    ...           2.0  NaN  2800.0     NaN   
1256691              49.0    NaN    ...           0.0  NaN  2500.0     NaN   
1256892             144.0  144.0    ...           0.0  0.0  2200.0   500.0   
1261181              49.0   49.0    ...           0.0  0.0  1200.0   240.0   
1261521             256.0  144.0    ...           4.0  0.0  2000.0  2000.0   

                  P47T          lnIncome           lnIncomeT            
familyRelation       1       2         1         2         1         2  
id                                                                      
1250971         4200.0     NaN  7.937375       NaN  8.342840       NaN  
1256691         6500.0     NaN  7.824046       NaN  8.779557       NaN  
1256892         2200.0   900.0  7.696213  6.214608  7.696213  6.802395  
1261181         1200.0  1195.0  7.090077  5.480639  7.090077  7.085901  
1261521         2000.0  2900.0  7.600902  7.600902  7.600902  7.972466  

[5 rows x 24 columns]

In [15]:
pivot.columns

MultiIndex(levels=[[u'age', u'age2', u'female', u'education', u'education2', u'primary', u'secondary', u'university', u'P21', u'P47T', u'lnIncome', u'lnIncomeT'], [1, 2]],
           labels=[[0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11], [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]],
           names=[None, u'familyRelation'])

In [17]:
pivot.to_csv('data/pivotInd.csv')

In [16]:
#para leer en otro archivo
dataN = pd.read_csv('data/pivotInd.csv', names = ['id','headAge','spouseAge','headAge2','spouseAge2',
                                                  'headFemale','spouseFemale','headEduc','spouseEduc',
                                                  'headEduc2','spouseEduc2','headPrimary','spousePrimary',
                                                  'headSecondary','spouseSecondary','headUniversity','spouseUniversity',
                                                  'headP21','spouseP21','headP47T','spouseP47T',
                                                  'headLnIncome','spouseLnIncome','headLnIncomeT','spouseLnIncomeT'
                                                 ],skiprows = 3)
dataN.head()
np.where(np.isnan(pivot.values),0,pivot.values)


,id,headAge,spouseAge,headAge2,spouseAge2,headFemale,spouseFemale,headEduc,spouseEduc,headEduc2,...,headUniversity,spouseUniversity,headP21,spouseP21,headP47T,spouseP47T,headLnIncome,spouseLnIncome,headLnIncomeT,spouseLnIncomeT
0,1250971,57.0,NaN,3249.0,NaN,1.0,NaN,14.0,NaN,196.0,...,2.0,NaN,2800.0,NaN,4200.0,NaN,7.937375,NaN,8.342840,NaN
1,1256691,61.0,NaN,3721.0,NaN,0.0,NaN,7.0,NaN,49.0,...,0.0,NaN,2500.0,NaN,6500.0,NaN,7.824046,NaN,8.779557,NaN
2,1256892,31.0,23.0,961.0,529.0,0.0,1.0,12.0,12.0,144.0,...,0.0,0.0,2200.0,500.0,2200.0,900.0,7.696213,6.214608,7.696213,6.802395
3,1261181,38.0,30.0,1444.0,900.0,0.0,1.0,7.0,7.0,49.0,...,0.0,0.0,1200.0,240.0,1200.0,1195.0,7.090077,5.480639,7.090077,7.085901
4,1261521,25.0,25.0,625.0,625.0,1.0,0.0,16.0,12.0,256.0,...,4.0,0.0,2000.0,2000.0,2000.0,2900.0,7.600902,7.600902,7.600902,7.972466


array([[   57.,     0.,  3249., ...,     0.,   196.,     0.],
       [   61.,     0.,  3721., ...,     0.,    49.,     0.],
       [   31.,    23.,   961., ...,    12.,   144.,   144.],
       ..., 
       [   43.,     0.,  1849., ...,     0.,   144.,     0.],
       [    0.,    66.,     0., ...,     7.,     0.,    49.],
       [   28.,     0.,   784., ...,     0.,    49.,     0.]])

In [12]:
pivot.shape

(1916, 10)